In [1]:
# rendering related
import os, imageio
import jax
import mujoco
from dm_control.mujoco import wrapper
from dm_control.mujoco.wrapper.mjbindings import enums
import pickle
from tqdm import tqdm

In [2]:
# loading data
with open("./../clips/all_snips.p", "rb") as f:
    all_traj = pickle.load(f)

# render overlay
scene_option = wrapper.MjvOption()
scene_option.geomgroup[2] = 1
scene_option.sitegroup[2] = 1

scene_option.sitegroup[3] = 1
scene_option.flags[enums.mjtVisFlag.mjVIS_TRANSPARENT] = True
scene_option.flags[enums.mjtVisFlag.mjVIS_LIGHT] = False
scene_option.flags[enums.mjtVisFlag.mjVIS_CONVEXHULL] = True
scene_option.flags[enums.mjtRndFlag.mjRND_SHADOW] = False
scene_option.flags[enums.mjtRndFlag.mjRND_REFLECTION] = False
scene_option.flags[enums.mjtRndFlag.mjRND_SKYBOX] = False
scene_option.flags[enums.mjtRndFlag.mjRND_FOG] = False
mj_model = mujoco.MjModel.from_xml_path(f"../assets/rodent.xml")

mj_model.opt.solver = {
    "cg": mujoco.mjtSolver.mjSOL_CG,
    "newton": mujoco.mjtSolver.mjSOL_NEWTON,
}["cg"]

mj_model.opt.iterations = 6
mj_model.opt.ls_iterations = 6
mj_model.opt.jacobian = 0  # dense
mj_data = mujoco.MjData(mj_model)

os.environ["MUJOCO_GL"] = "egl"

renderer = mujoco.Renderer(mj_model, height=512, width=512)
mujoco.mj_kinematics(mj_model, mj_data)

In [3]:
len(all_traj["snips_order"])

842

In [9]:
all_traj["qpos"].shape

(210500, 74)

In [4]:
qpos_clips = all_traj["qpos"].reshape((-1, 250, 74))

2024-07-02 18:24:56.833491: W external/xla/xla/service/gpu/nvptx_compiler.cc:765] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.5.40). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


In [5]:
all_traj["snips_order"]

['././snippets_2_25_2021/snips/FastWalk_171.p',
 '././snippets_2_25_2021/snips/LGroom_158.p',
 '././snippets_2_25_2021/snips/FastWalk_169.p',
 '././snippets_2_25_2021/snips/FastWalk_44.p',
 '././snippets_2_25_2021/snips/RGroom_45.p',
 '././snippets_2_25_2021/snips/FastWalk_58.p',
 '././snippets_2_25_2021/snips/Rear_155.p',
 '././snippets_2_25_2021/snips/RGroom_16.p',
 '././snippets_2_25_2021/snips/Rear_82.p',
 '././snippets_2_25_2021/snips/Walk_145.p',
 '././snippets_2_25_2021/snips/Rear_175.p',
 '././snippets_2_25_2021/snips/RGroom_164.p',
 '././snippets_2_25_2021/snips/Walk_115.p',
 '././snippets_2_25_2021/snips/Walk_98.p',
 '././snippets_2_25_2021/snips/Rear_104.p',
 '././snippets_2_25_2021/snips/Rear_166.p',
 '././snippets_2_25_2021/snips/Walk_174.p',
 '././snippets_2_25_2021/snips/Rear_158.p',
 '././snippets_2_25_2021/snips/LGroom_134.p',
 '././snippets_2_25_2021/snips/Rear_5.p',
 '././snippets_2_25_2021/snips/Rear_135.p',
 '././snippets_2_25_2021/snips/FastWalk_87.p',
 '././snipp

In [8]:
qpos_clips

XlaRuntimeError: INVALID_ARGUMENT: Failed to initialize gpu stream

In [13]:
all_traj["snips_order"][0].split("/")[-1].split(".")[0]

'FastWalk_171'

In [14]:
for i, name in enumerate(tqdm(all_traj["snips_order"])):
    frames = []
    # render while stepping using mujoco
    name = name.split("/")[-1].split(".")[0]
    video_path = f"videos/{name}.mp4"
    with imageio.get_writer(video_path, fps=50) as video:
        for qpos in tqdm(qpos_clips[i], leave=False):
            # Set keypoints
            mj_data.qpos = qpos
            mujoco.mj_forward(mj_model, mj_data)
            renderer.update_scene(mj_data, camera="close_profile")
            pixels = renderer.render()
            video.append_data(pixels)
            frames.append(pixels)

  0%|          | 0/842 [00:00<?, ?it/s]/root/anaconda3/envs/vnl/lib/python3.11/subprocess.py:1885: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = _fork_exec(
100%|██████████| 842/842 [4:37:43<00:00, 19.79s/it]  


# Good Clip to Imitate

- FastWalk_61
- FastWalk_65
- FastWalk_80
- FastWalk_93
- Walk_20
- Walk_27
- Walk_34
- Walk_49

In [ ]:
import multiprocessing

def task(qposes_ref, qposes_rollout):
    # Render the walker with the reference expert demonstration trajectory
    os.environ["MUJOCO_GL"] = "osmesa"

    # render overlay
    scene_option = wrapper.MjvOption()
    scene_option.geomgroup[2] = 1
    scene_option.sitegroup[2] = 1
    scene_option.sitegroup[3] = 1
    scene_option.flags[enums.mjtVisFlag.mjVIS_TRANSPARENT] = True
    scene_option.flags[enums.mjtVisFlag.mjVIS_LIGHT] = False
    scene_option.flags[enums.mjtVisFlag.mjVIS_CONVEXHULL] = True
    scene_option.flags[enums.mjtRndFlag.mjRND_SHADOW] = True
    scene_option.flags[enums.mjtRndFlag.mjRND_REFLECTION] = False
    scene_option.flags[enums.mjtRndFlag.mjRND_SKYBOX] = False
    scene_option.flags[enums.mjtRndFlag.mjRND_FOG] = False
    mj_model = mujoco.MjModel.from_xml_path(
        f"./assets/{env_cfg[train_config.env_name]['rendering_mjcf']}"
    )

    mj_model.opt.solver = {
        "cg": mujoco.mjtSolver.mjSOL_CG,
        "newton": mujoco.mjtSolver.mjSOL_NEWTON,
    }["cg"]

    mj_model.opt.iterations = 6
    mj_model.opt.ls_iterations = 6
    mj_model.opt.jacobian = 0  # dense
    mj_data = mujoco.MjData(mj_model)

    # save rendering and log to wandb
    os.environ["MUJOCO_GL"] = "osmesa"
    mujoco.mj_kinematics(mj_model, mj_data)
    renderer = mujoco.Renderer(mj_model, height=512, width=512)

    frames = []
    # render while stepping using mujoco
    video_path = f"{model_path}/{num_steps}.mp4"

    print("saving videos...")

    with imageio.get_writer(video_path, fps=float(1.0 / eval_env.dt)) as video:
        for qpos1, qpos2 in zip(qposes_ref, qposes_rollout):
            mj_data.qpos = np.append(qpos1, qpos2)
            mujoco.mj_forward(mj_model, mj_data)

            renderer.update_scene(
                mj_data, camera=f"{env_cfg[train_config.env_name]['camera']}"
            )

            pixels = renderer.render()
            video.append_data(pixels)
            frames.append(pixels)

    wandb.log({"eval/rollout": wandb.Video(video_path, format="mp4")})

# process it in another separate thread
process = multiprocessing.Process(
    target=task, args=(qposes_ref, qposes_rollout)
)
process.start()

Background task running: Alice